In [ ]:
import traceback
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from tkinter.simpledialog import askstring
import threading
from dealroadshow_scraper import (
    get_user_input,
    initialize_driver,
    login_to_dealroadshow,
    wait_for_interaction,
    pause_video,
    navigate_to_first_slide,
    get_nb_slides,
    create_output_folders,
    take_screenshots,
    save_screenshots_to_word,
    save_screenshots_to_pdf,
    cleanup_files
)

class InputWindow:
    def __init__(self, root):
        self.root = root
        self.root.title("DealRoadshow Screenshot App")

        # Create and place widgets
        self.create_widgets()

    def create_widgets(self):
        # Company Name Entry
        ttk.Label(self.root, text="Company Name:").place(relx=0.5, rely=0.05, anchor="center")
        self.company_name_var = tk.StringVar()
        ttk.Entry(self.root, textvariable=self.company_name_var, width=30).place(relx=0.5, rely=0.1, anchor="center")

        # Deal Entry Code Entry
        ttk.Label(self.root, text="Deal Entry Code:").place(relx=0.5, rely=0.2, anchor="center")
        self.deal_entry_code_var = tk.StringVar()
        ttk.Entry(self.root, textvariable=self.deal_entry_code_var, width=30).place(relx=0.5, rely=0.25, anchor="center")

        # Email Dropdown
        ttk.Label(self.root, text="NetRoadshow Email Address:").place(relx=0.5, rely=0.35, anchor="center")
        self.email_var = tk.StringVar()
        self.email_combobox = ttk.Combobox(self.root, textvariable=self.email_var)
        email_addresses = ["e.desinety@sycomore-am.com", "tony.lebon@sycomore-am.com", "alex.mory@sycomore-am.com"]
        self.email_combobox['values'] = email_addresses
        self.email_combobox.config(width=max(len(email) for email in email_addresses))
        self.email_combobox.place(relx=0.5, rely=0.4, anchor="center")
        self.email_combobox.set("Select Email")

        # Mode Selection
        ttk.Label(self.root, text="Mode:").place(relx=0.5, rely=0.5, anchor="center")
        self.mode_var = tk.IntVar(value = 2)
        ttk.Radiobutton(self.root, text="Debug Mode", variable=self.mode_var, value=1).place(relx=0.33, rely=0.55, anchor="center")
        ttk.Radiobutton(self.root, text="User Mode", variable=self.mode_var, value=2).place(relx=0.66, rely=0.55, anchor="center")

        # Start Button
        self.start_button = ttk.Button(self.root, text="Start", command=self.start_process)
        self.start_button.place(relx=0.5, rely=0.65, anchor="center")

    def start_process(self):
        company_name = self.company_name_var.get()
        deal_entry_code = self.deal_entry_code_var.get()
        email = self.email_var.get()
        mode = self.mode_var.get()

        if not company_name or not deal_entry_code or not email:
            messagebox.showerror("Input Error", "All fields are required.")
            return

        # Hide the current window and open the log window
        self.root.withdraw()
        log_window = LogWindow(self.root, self, company_name, deal_entry_code, email, mode)
        log_window.start_process()

    def close_app(self):
        self.root.destroy()
        self.root.quit()

class LogWindow:
    def __init__(self, parent_root, parent, company_name, deal_entry_code, email, mode):
        self.parent_root = parent_root
        self.parent = parent
        self.root = tk.Toplevel(parent_root)
        self.root.title("Status Log")
        self.root.minsize(400,400)
        self.root.maxsize(400,400)

        self.company_name = company_name
        self.deal_entry_code = deal_entry_code
        self.email = email
        self.mode = mode
    
        # Create and place widgets
        self.create_widgets()

    def create_widgets(self):
        # Status Log
        self.status_frame = tk.Frame(self.root)
        self.status_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        self.status_log = tk.Text(self.status_frame, height=15, width=45, wrap=tk.WORD, font=("Helvetica", 10))
        self.status_log.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.status_log.insert(tk.END, "Status Log:\n")

        self.scrollbar = tk.Scrollbar(self.status_frame, command=self.status_log.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.status_log.config(yscrollcommand=self.scrollbar.set)

        # Progress Bar Frame
        self.progress_frame = tk.Frame(self.root)
        self.progress_frame.pack(fill=tk.X, padx=10, pady=5)

        # Progress Bar
        self.progress = ttk.Progressbar(self.progress_frame, orient="horizontal", length=300, mode="determinate")
        self.progress.pack(side=tk.LEFT, padx=5)

        self.progress_label = ttk.Label(self.progress_frame, text="0 slides")
        self.progress_label.pack(side=tk.RIGHT, padx=5)
    
    def append_log(self, message):
        self.status_log.insert(tk.END, message + '\n')
        self.status_log.yview(tk.END)

    def start_process(self):
        threading.Thread(target=self.run_process).start()

    def update_progress(self, value, maximum):
        self.progress['value'] = value
        self.progress['maximum'] = maximum
        self.progress_label.config(text=f"{value}/{maximum} slides")
        self.root.update_idletasks()

    def run_process(self):
        
        def log(message):
            self.append_log(message)
        try:  
            log("Initializing WebDriver...")
            driver = initialize_driver()
            log("Logging into Roadshow...")
            if login_to_dealroadshow(driver, self.email, self.deal_entry_code):
                if wait_for_interaction(driver):
                    pause_video(driver)
                    log("Navigating to first slide...")
                    navigate_to_first_slide(driver)
                    log("Getting number of slides...")
                    nb_slides = get_nb_slides(driver)
                    self.update_progress(0, nb_slides)
                    log("Creating output folders...")
                    screenshots_folder, output_folder = create_output_folders(self.company_name)
                                
                    log("Taking screenshots...")
                    def screenshot_callback(progress):
                        self.update_progress(progress, nb_slides)
        
                    screenshot_paths = take_screenshots(driver, screenshots_folder, nb_slides, screenshot_callback)
                
                    driver.quit()
            
                    log("Saving screenshots to Word document...")
                    doc_path = save_screenshots_to_word(screenshot_paths, output_folder, self.company_name)
            
                    log("Saving Word document as PDF...")
                    pdf_path = save_screenshots_to_pdf(doc_path)
            
                    log("Cleaning up files...")
                    cleanup_files(output_folder, pdf_path, self.mode)
                    
                    messagebox.showinfo("Process Completed", "The work is saved, you can close this window.")
                    self.parent.close_app()
                    self.parent.root.destroy()
                else:
                    messagebox.showerror("Interaction Error", "Failed to complete required interactions.")
                    driver.quit()
                    self.parent_root.deiconify()
                    self.root.destroy()
            else:
                messagebox.showerror("Login Error", "Failed to login, check credentials or NRS expiration.")
                driver.quit()
                self.parent_root.deiconify()
                self.root.destroy()
        except Exception as e:
            error_message = str(e)
            error_traceback = traceback.format_exc()
            log(f"Error: {error_message}\n{error_traceback}")
            messagebox.showerror("Process Error", f"An error occurred: {error_message}")
            driver.quit()
            self.parent_root.deiconify()
            self.root.destroy()

# Create the main window
root = tk.Tk()
root.minsize(400,400)
root.maxsize(400,400)
app = InputWindow(root)
root.mainloop()

In [1]:
import 